In [1]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
seed = 1234

In [2]:
df = pd.read_csv('preprocessing_kfold.csv')
df.head(10)

,Date,User,Isi,Sentiment,cleartweet
0,2023-02-06 05:07:39+00:00,ridwanibnuhasan,@BigAlphaID Sesuai dengan target. Dan mungkin ...,netral,sesuai target dunia ancam resesi
1,2023-02-06 04:59:29+00:00,faliqhan,@BigAlphaID Rasanya untuk nyemplung sampe minu...,negatif,nyemplung sampe minus mustahil sih sampe q unt...
2,2023-02-06 04:56:21+00:00,COREIndonesia,Lalu bagaimana dengan tahun ini? meskipun dipr...,netral,prediksi tumbuh lambat core indonesia p
3,2023-02-06 04:48:33+00:00,Windutmiy,@garudadidada10 ancaman resesi global yang mem...,netral,ancam resesi global butuh bijak fiskal moneter...
4,2023-02-06 04:29:56+00:00,dynarfitrianti1,@garudadidada10 kita pasti mampu melewati rese...,positif,lewat resesi global
5,2023-02-06 04:29:02+00:00,catatankaqihati,RT @Anggita_lung: Dengan pencabutan status PPK...,positif,cabut status ppkm optimis kondisi ekonomi indo...
6,2023-02-06 04:22:32+00:00,soloposdotcom,Resesi Global Disebut Tak Terlalu Berdampak ke...,netral,resesi global dampak ri respons rhenald kasali
7,2023-02-06 04:17:59+00:00,DdMulyadi2,RT @CoklatBenkBenk: Harga Minyak Diprediksi Je...,negatif,harga minyak prediksi jeblok usd barel analis ...
8,2023-02-06 04:12:19+00:00,Indiah752834981,Itu bukan prestasi nomor 1 didunia tapi byk ne...,negatif,prestasi nomor dunia byk negara resesi
9,2023-02-06 04:09:38+00:00,Di0nAngelo,RT @CoklatBenkBenk: Harga Minyak Diprediksi Je...,negatif,harga minyak prediksi jeblok usd barel analis ...


In [3]:
df = df.astype({'Sentiment' : 'category'})
df = df.astype({'cleartweet' : 'object'})
df.dtypes

Date            object
User            object
Isi             object
Sentiment     category
cleartweet      object
dtype: object

In [4]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

tvec= CountVectorizer()
X_cVec = tvec.fit_transform(df['cleartweet'].values.astype('U'))
print(X_cVec)
h_tfidf = TfidfTransformer()
x_tfidf = h_tfidf.fit_transform(X_cVec)
print(x_tfidf)
X = df.cleartweet
Y = df.Sentiment

  (0, 1379)	1
  (0, 1499)	1
  (0, 405)	1
  (0, 70)	1
  (0, 1282)	1
  (1, 1282)	1
  (1, 1069)	1
  (1, 1328)	2
  (1, 962)	1
  (1, 995)	1
  (1, 1391)	1
  (1, 1612)	1
  (1, 536)	1
  (1, 1599)	1
  (1, 590)	1
  (1, 1060)	1
  (1, 631)	1
  (2, 1213)	1
  (2, 1588)	1
  (2, 822)	1
  (2, 306)	1
  (2, 604)	1
  (3, 70)	1
  (3, 1282)	1
  (3, 509)	1
  :	:
  (1003, 1174)	1
  (1003, 1296)	1
  (1003, 1537)	1
  (1003, 455)	1
  (1003, 1563)	1
  (1003, 105)	1
  (1003, 472)	1
  (1003, 307)	1
  (1004, 1282)	4
  (1004, 1552)	1
  (1004, 450)	1
  (1004, 609)	1
  (1004, 568)	1
  (1004, 1509)	1
  (1004, 446)	1
  (1005, 1282)	1
  (1005, 1069)	1
  (1005, 1328)	2
  (1005, 962)	1
  (1005, 995)	1
  (1005, 1391)	1
  (1005, 1612)	1
  (1005, 536)	1
  (1005, 1599)	1
  (1005, 590)	1
  (0, 1499)	0.5925785535800462
  (0, 1379)	0.6481366492939087
  (0, 1282)	0.12324821914195506
  (0, 405)	0.3611834320405549
  (0, 70)	0.2883157064344359
  (1, 1612)	0.3026011120535862
  (1, 1599)	0.23421316144324364
  (1, 1391)	0.249150823020351

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE

kf = KFold(n_splits=7, shuffle=True, random_state=40)
X_array = x_tfidf.toarray()
def cross_val(estimator):
    acc = []
    pcs = []
    rec = []

    for train_index, test_index in kf.split(X_array, Y):
        X_train, X_test = X_array[train_index], X_array[test_index]
        y_train, y_test = Y[train_index], Y[test_index]

        sm = SMOTE()
        X_train_oversampled, y_train_oversampled = sm.fit_resample(X_train, y_train)
        model = estimator.fit(X_train_oversampled, y_train_oversampled)
        y_pred=model.predict(X_test)

        acc.append(accuracy_score(y_test, y_pred))
        pcs.append(precision_score(y_test, y_pred, average='macro', zero_division=0))
        rec.append(recall_score(y_test, y_pred, average='macro', zero_division=0))
        rec.append(f1_score(y_test, y_pred, average='macro', zero_division=0))

        print(classification_report(y_test, y_pred, zero_division=0))
        print(f'confusion matrix:\n {confusion_matrix(y_test, y_pred)}')
        print('==================================================\n')

    print(f'rata-rata Akurasi: {np.mean(acc)}')
    print(f'rata-rata Precision: {np.mean(pcs)}')
    print(f'rata-rata Recall: {np.mean(rec)}')
    print(f'rata-rata f1-score: {np.mean(rec)}')

from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
cross_val(nb)

              precision    recall  f1-score   support

     negatif       0.79      0.85      0.82        53
      netral       0.86      0.68      0.76        72
     positif       0.43      0.68      0.53        19

    accuracy                           0.74       144
   macro avg       0.69      0.74      0.70       144
weighted avg       0.78      0.74      0.75       144

confusion matrix:
 [[45  3  5]
 [11 49 12]
 [ 1  5 13]]

              precision    recall  f1-score   support

     negatif       0.81      0.79      0.80        58
      netral       0.88      0.73      0.80        59
     positif       0.47      0.67      0.55        27

    accuracy                           0.74       144
   macro avg       0.72      0.73      0.72       144
weighted avg       0.77      0.74      0.75       144

confusion matrix:
 [[46  3  9]
 [ 5 43 11]
 [ 6  3 18]]

              precision    recall  f1-score   support

     negatif       0.74      0.76      0.75        55
      netral   